<a href="https://www.kaggle.com/code/syerramilli/ps3e22-eda-catboost-baseline?scriptVersionId=143811280" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import chi2_contingency # for association between different categorical variables

from numbers import Number 
from pathlib import Path
from typing import Optional, Dict

plt.style.use('ggplot')

In [ ]:
path = Path('/kaggle/input/playground-series-s3e22')
train = pd.read_csv(path/'train.csv',index_col=['id'])
test = pd.read_csv(path/'test.csv',index_col=['id'])

del train['hospital_number']
del test['hospital_number']

train.head()

In [ ]:
print(f'Number of rows in training set: {train.shape[0]}')
print(f'Number of rows in test set: {test.shape[0]}')

Here's a quick snapshot of the training data. From this snapshot, we can see the following
1. There are 26 potential feature columns. 
2. There are several numerical features (indicated by dtypes `float64` and `int64` dtypes). None of them have missing entries.
3. There are several features with dtype - `object`. These are likely categorical valued, although some of them like `age` may be processed as numerical. 
4. Among the columns with the `object` dtype, there are a few entries with missing values.

In [ ]:
# quick snapshot
train.info()

## Target

The goal of this task to predict `outcome` - whether the horse survived or not - based on past medical conditions. There are three classes: lived, died, and euthanized. In the cell below, we show the bar plot of the counts for each class. 

In [ ]:
train['outcome'].value_counts().plot(kind='barh')

## Lesions 

From the description in the [original dataset](https://www.kaggle.com/datasets/yasserh/horse-survival-dataset), the columns `lesion_1`, `lesion_2` and `lesion_3` encode the site, type, subtype, and specific code.

In [ ]:
for i in range(1,4):
    column = f'lesion_{i}'
    print(f"Number of unique values in {column}: {train[column].nunique()}")

It turns out almost all the entires of `lesion_2` and `lesion_3` are 0. Therefore, we will drop them in the remaining analysis.

In [ ]:
for i in range(2,4):
    print(train[f'lesion_{i}'].value_counts())

In [ ]:
# dropping lesion_2 and lesion_3
train = train.drop(['lesion_2','lesion_3'],axis=1)
test = test.drop(['lesion_2','lesion_3'],axis=1)

### Lesion site

In [ ]:
def map_lesion_site(value:str) -> str:
    if value[:2] == "11" and len(value) == 5:
        return "all_intestinal"
    elif value[0] == "1":
        return "gastric"
    elif value[0] == "2":
        return "sm_intestine"
    elif value[0] == "3":
        return "lg_colon"
    elif value[0] == "4":
        return "lg_colon_and_cecum"
    elif value[0] == "5":
        return "cecum"
    elif value[0] == "6":
        return "transverse_colon"
    elif value[0] == "7":
        return "retum_colon"
    elif value[0] == "8":
        return "uterus"
    elif value[0] == "9":
        return "bladder"
    elif value[0] == "0":
        return "none"
    else:
        return "ERROR"
    
train['lesion_site'] = train['lesion_1'].astype(str).apply(map_lesion_site)
test['lesion_site'] = test['lesion_1'].astype(str).apply(map_lesion_site)

fig,ax = plt.subplots(1, 1, figsize=(6,4))
train['lesion_site'].value_counts().plot(kind='bar', ax=ax)
_ = ax.tick_params(axis='x', rotation=60)
# to be used later
site_order = [text.get_text() for text in ax.get_xticklabels()]

In [ ]:
ax = (
    train
    .groupby('lesion_site')['outcome']
    .value_counts(normalize=True)
    .mul(100)
    .rename('Percentage')
    .reset_index()
    .pipe(
        (sns.catplot,'data'), y='lesion_site',x='Percentage',hue='outcome',
        order= site_order,
        hue_order=['died', 'euthanized', 'lived'],
        kind='bar', height=5, aspect=7/5
    )
)

### Lesion type


In [ ]:
def map_lesion_type(value:str) -> str:
    if value == '0':
        return "none"
    
    value2 = value[2] if len(value)==5 else value[1]
    
    if value2 == '1':
        return "simple"
    elif value2 == '2':
        return 'strangulation'
    elif value2 == '3':
        return 'inflammation'
    elif value2 == '4':
        return 'other'
    
    return 'ERROR'

train['lesion_type'] = train['lesion_1'].astype(str).apply(map_lesion_type)
test['lesion_type'] = test['lesion_1'].astype(str).apply(map_lesion_type)

fig,ax = plt.subplots(1, 1, figsize=(6,4))
train['lesion_type'].value_counts().plot(kind='bar', ax=ax)
_ = ax.tick_params(axis='x', rotation=60)
# to be used later
type_order = [text.get_text() for text in ax.get_xticklabels()]

In [ ]:
ax = (
    train
    .groupby('lesion_type')['outcome']
    .value_counts(normalize=True)
    .mul(100)
    .rename('Percentage')
    .reset_index()
    .pipe(
        (sns.catplot,'data'), y='lesion_type',x='Percentage',hue='outcome',
        order= type_order,
        hue_order=['died', 'euthanized', 'lived'],
        kind='bar', height=4, aspect=6/4
    )
)
#_ = ax.tick_params(axis='x', rotation=30)

**TODO**: Decode the lesion subtype and code. 

In [ ]:
# delete lesion_1
del train['lesion_1']
del test['lesion_1']

## Numerical Features

In [ ]:
numerical_cols= train.select_dtypes(include=['number']).columns.tolist()
print(f'Number of numerical columns: {len(numerical_cols)}')

We plot the histograms of the 10 numerical features in the cell below.  The features `respiratory_rate`, `total_protein`, and `abdomo_protein`  have positive skew.

In [ ]:
n_rows = 2
n_cols = 4
fig,axs = plt.subplots(n_rows,n_cols,figsize=(4*n_cols,3*n_rows))
for i in range(n_rows):
    for j in range(n_cols):
        col_index = n_cols*i+j
        if col_index == 7:
            break
        _ = sns.histplot(data=train,x=numerical_cols[col_index], ax=axs[i,j],bins=20)
        
fig.tight_layout()

To confirm our observations about the skew in the distributions in some of the features, we compute the skewness statistic for the remaining 8 numerical features.

In [ ]:
skewness = train[numerical_cols].skew().sort_values(ascending=False)
skewness

We now plot the boxplots of the features grouped by the different classes. For the 4 features with significant positive skew, we apply a log transform before generating the boxplot. Some observations:

1. The horses that lived generally had a lower `pulse`  than the other groups.
2. They also had a lower `packed_cell_volume` than the other groups
3. Horses that were euthanized had orders of magnitude higher `total_protein` than the other two groups, although there are quite a few outliers in the other groups.

In [ ]:
n_rows = 2
n_cols = 4
fig,axs = plt.subplots(n_rows,n_cols,figsize=(3*n_cols,3*n_rows))
for i in range(n_rows):
    for j in range(n_cols):
        col_index = n_cols*i+j
        if col_index == 7:
            break
        
        
        column = numerical_cols[col_index]
        
        if skewness.loc[column] > 1:
            _ = sns.boxplot(y=np.log(train[column]),x=train['outcome'],ax=axs[i,j])
            _ = axs[i,j].set_ylabel(f'log({column})')
        else:
            _ = sns.boxplot(data=train,y=column,x='outcome', ax=axs[i,j])
        
fig.tight_layout()

Here is the histogram plot of the features grouped by outcome. For the 4 features with significant positive skew, we apply a log transform before generating the boxplot. Some observations:

In [ ]:
n_rows = 2
n_cols = 4
fig,axs = plt.subplots(n_rows,n_cols,figsize=(5*n_cols,3*n_rows))
for i in range(n_rows):
    for j in range(n_cols):
        col_index = n_cols*i+j
        if col_index == 7:
            break
        
        
        column = numerical_cols[col_index]
        
        if skewness.loc[column] > 1:
            _ = sns.histplot(x=np.log(train[column]), hue=train['outcome'],ax=axs[i,j], bins=20, stat='density', kde=True)
            _ = axs[i,j].set_xlabel(f'log({column})')
        else:
            _ = sns.histplot(data=train,x=column,hue='outcome', ax=axs[i,j], bins=20, stat='density', kde=True)
            
        if col_index > 0:
            _ = axs[i,j].get_legend().remove()
        
fig.tight_layout()

Finally, we plot the correlation heatmap, where we compute the Spearman rank correlation. There are no red flags here.

In [ ]:
corr_matrix = train[numerical_cols].corr(method='spearman')
mask =np.triu(np.ones_like(corr_matrix, dtype=bool))
fig,ax = plt.subplots(1,1,figsize=(5,4),dpi=150)
_ = sns.heatmap(corr_matrix,annot=True,fmt='.2f',mask=mask,ax=ax)
_ = ax.set_facecolor('w')


## Numerical feature engineering

In [ ]:
train['log_pulseSq_total_protein'] = -np.log(train['total_protein']) + 2*np.log(train['pulse'])
test['log_pulseSq_total_protein'] = -np.log(test['total_protein']) + 2*np.log(test['pulse'])


_ = sns.histplot(data=train, x='log_pulseSq_total_protein',hue='outcome', kde=True, stat='density')

## Categorical features

In [ ]:
rem_columns = train.drop('outcome',axis=1).select_dtypes(include=['object']).columns.tolist()
print(f'Number of columns with dtype object: {len(rem_columns)}')

We will exlcude any column where the mode (aka most common value) occurs in more than 85% of the entries.

In [ ]:
def get_mode_fraction(x:pd.Series) -> float:
    cts = x.value_counts(sort=True, ascending=False)
    return cts.iloc[0]/x.shape[0]

for i, column in enumerate(rem_columns):
    mode_frac = get_mode_fraction(train[column])
    if mode_frac > 0.85:
        # drop the feature if >85% of the observations 
        # belong to the mode
        print(f'Dropping {column} with the mode having {mode_frac*100:.2f}% observations')
        
        del train[column]
        del test[column]
        
        rem_columns.pop(i)

We now run chi-squared contigency tests to test the significance of the relationships of each categorical feature with the response. It appears that all 16 features have significant relationship with `outcome`.

In [ ]:
def contingency_test(input_col:str, significance_level:float=0.01) -> bool:
    stat,pval,_,_ = chi2_contingency(pd.crosstab(train[input_col], train['outcome']))
    
    return abs(stat), pval < significance_level

chi2_tests_df = pd.DataFrame(
    [contingency_test(column) for column in rem_columns],
    index=rem_columns,
    columns=['abs_stat', 'is_significant']
).sort_values(by=['is_significant', 'abs_stat'], ascending=False)

print(f'Number of categorical features with signficant relationship with outcome: {chi2_tests_df["is_significant"].sum()}')

In the cell below, we generate a bar plots of the counts of the categories for each of these features, *except the two lesion features decoded earlier*. Within each category, we separate the counts by class. Some preliminary observations:

1. For all the features, the class counts seem to differ between atleast two categories, suggesting some relationship.
2. Some of the features like `temp_of_extremities`can be encoded as ordinal integers. This can help reduce the dimensionaloity.
3. For a lot of features, some of the categories have very few observations. We might need to merge these categories to learn something useful. 

In [ ]:
n_rows = 5
n_cols = 3
fig,axs = plt.subplots(n_rows,n_cols,figsize=(5*n_cols,4*n_rows))
for i in range(n_rows):
    for j in range(n_cols):
        column = rem_columns[n_cols*i+j]
        _ = sns.countplot(data=train,x=column,hue='outcome', ax=axs[i,j])
        _ = axs[i,j].tick_params(axis='x', rotation=30)
        
fig.tight_layout()

**TODO**: Merge categories of the `lesion_site` feature so that the model can learn something useful for categories with very few observations.

In [ ]:
categorical_columns = [
    'mucous_membrane', 'abdomen','rectal_exam_feces', 
    'lesion_site', 'lesion_type' 
]

def preprocess_categorical(df:pd.DataFrame) -> None:
    # cleaning some of the categorical features
    df['peristalsis'] = df['peristalsis'].replace('distend_small',pd.NA)
    df['rectal_exam_feces'] = df['rectal_exam_feces'].replace('serosanguious',pd.NA)
    
    # merging some of the categories
    df['capillary_refill_time'] = df['capillary_refill_time'].replace('3','more_3_sec')
    df['pain'] = df['pain'].replace('slight','alert')
    
    
    # encoding some of the catgeorical level 
    ordinal_and_binary_dict = {
        'surgery': ['no','yes'], 
        'temp_of_extremities': ['cold','cool', 'normal', 'warm'], 
        'peripheral_pulse': ['absent','reduced', 'normal','increased'], 
        'pain':['alert', 'depressed', 'mild_pain', 'moderate', 'severe_pain', 'extreme_pain'],
        'capillary_refill_time': ['less_3_sec', 'more_3_sec'], 
        'peristalsis': ['absent', 'hypomotile', 'normal', 'hypermotile'], 
        'abdominal_distention': ['none', 'slight', 'moderate', 'severe'], 
        'nasogastric_tube': ['none', 'slight', 'significant'], 
        'nasogastric_reflux': ['none','slight','less_1_liter', 'more_1_liter'], 
        'abdomo_appearance': ['serosanguious', 'cloudy', 'clear'], 
        'surgical_lesion': ['no', 'yes'], 
        'cp_data': ['no', 'yes']
    }
    
    for column, levels in ordinal_and_binary_dict.items():
        df[column] = df[column].replace({
            level:i for i,level in enumerate(levels)
        })
        
    for column in categorical_columns:
        # useful for other featur
        df[column] = df[column].astype('category')
        

# modify columns in place
preprocess_categorical(train)
preprocess_categorical(test)

## Missing values

In the quick snapshot earlier, we found that some of the features (encoded as `object`) have some values missing. In the cell below, I compute the fraction of missing values in each column. (Note: Columns with no missing values are excluded).

In [ ]:
def filter_greater_than(series:pd.Series,threshold:Number) -> pd.Series:
    '''
    Returns series elements greater than threshold. This funtion can be
    used with the .pipe methods
    '''
    return series[series>threshold]

def get_perc_missing(df:pd.DataFrame) -> pd.Series:
    return (
        (df.isnull().sum()/df.shape[0]*100)
        .sort_values(ascending=False)
        .pipe(filter_greater_than,threshold=0)
        .round(3)
    )

perc_missing = get_perc_missing(train)
perc_missing

The same columns have missing entries in the test set too, and the percentage of missing values in the test set is roughly the same. So, we will need a concrete imputation strategy.

In [ ]:
perc_missing_test = get_perc_missing(test)
assert perc_missing.shape[0] == perc_missing_test.shape[0]
perc_missing_test

For `abdomen` and `rectal_exam_feces`, we add a new category called `"missing"` for the missing entries.

In [ ]:
for column in ['abdomen','rectal_exam_feces']:
    train[column] = train[column].astype('object').fillna('missing').astype('category')
    test[column] = test[column].astype('object').fillna('missing').astype('category')

For the remaining columns, we impute the missing value with the mode. 

TODO: Use a more systematic imputation strategy.

In [ ]:
for column in perc_missing.iloc[2:].index:
    mode_col = train[column].mode().iloc[0]
    train[column] = train[column].fillna(mode_col)
    test[column] = test[column].fillna(mode_col)

## Catboost model

The function `fit_model` in the cell below, trains a catboost classification model that uses bagging for the individual trees. The function also allows the specification of hyperparameters as a dictionary through the `config` argument. If `config` is not specified, default values for the hyperparameters are used.

In [ ]:
def fit_model(
    X:pd.DataFrame,
    y:np.ndarray,
    config:Optional[Dict]=None,
    n_jobs:int=1,
    verbose:int=0,
    random_seed:int=100,
) -> CatBoostClassifier:
    '''
    Train a catboost classifier
    '''
    model = CatBoostClassifier(
        iterations = 500,
        thread_count = n_jobs,
        bootstrap_type = 'Bernoulli',
        subsample = 0.8,
        random_seed = random_seed,
        verbose = verbose
    )
    
    if config:
        # if config is supplied, set the model hyperparameters
        model.set_params(**config)
        
    cat_features = [
        column for column in X.columns if X[column].dtype == 'category'
    ]
        
    return model.fit(X, y, cat_features= cat_features)


In [ ]:
X = train.drop('outcome',axis=1)
le = LabelEncoder()
y = le.fit_transform(train['outcome'].values)

model = fit_model(X, y, n_jobs=4, verbose=50, random_seed=100)
model.save_model('baseline.cbm',format='cbm')

### Cross-validation

To provide a numerical measure for the baseline, we will use the f1score estimate from 4 replicates of 10-fold stratified cross-validation. We use replicated CV here since the size of the training set is small.

In [ ]:
import warnings
from tqdm import tqdm
def fit_and_test_fold(X, y, train_index,test_index) -> float:
    X_train = X.iloc[train_index,:];X_test = X.iloc[test_index,:]
    y_train = y[train_index]; y_test = y[test_index]
    
    # fit model on training data
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = fit_model(X_train, y_train, n_jobs=4)
    
    # generate predictions on test data
    test_pred = model.predict(X_test)
    
    return f1_score(y_test, test_pred, average='micro')


cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
cv_f1_scores = [None]*40
for i, (train_index, test_index) in tqdm(enumerate(cv.split(X,y))):
    cv_f1_scores[i] = fit_and_test_fold(X, y, train_index, test_index)

cv_f1 = np.mean(cv_f1_scores)
print(f'CV F1 for baseline model: {cv_f1:.3f}')

In [ ]:
# CV F1 for each replicate of 10-fold CV
# Clearly, there is some variability across replicates
np.array(cv_f1_scores).reshape(-1,10).mean(-1)

### Feature importances



We now compute the gain based feature importance measures from the catboost model.

**Notes**:
1. Feature importance measures from tree based models can be misleading.
2. In catboost, the default feature importance measure is based on the total gain from splits involving the feature.

In [ ]:
# gain based feature importances - not necessarily the most reliable
feat_imp = pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True)
feat_imp.plot(kind='barh')

## Feature importances through SHAP

The default feature importances computed by catboost (or any tree based model) can be misleading. Here, we will use SHAP measures to check the importance of each feature.

SHAP values represent the impact of each feature on the model's output for a specific instance. In multiclass classification, we will have a **separate** set of SHAP values for each class. These values tell us how each feature contributes to each class prediction, i.e., distinguishing the specific class from the rest.

In [ ]:
import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

In the SHAP summary plot, we plot a horizontal bar plot of the absolute SHAP value for each feature averaged across the observations.
Features with longer bars have a higher influence on the model's output for the specific class. Since we have 3 classes here, we will see 3 stacked bars for each feature. The features are ordered according to the cumulative length of the 3 bars. 

Let's look at the top 3 features from the plot below. 

1. `lesion_type` is very important for predicting "lived" outcomes,  and moderately important for predicting "died" outcomes and "euthanized" outcomes.
2. `total_protein` is very important for predicting "died" and "euthanized" outcomes, but not important for predicting "lived" outcomes.
3. `pain` is important for predicting "lived" and "died" outcomes, but not important for predicting "euthanized" outcomes.

In [ ]:
# Average of SHAP value magnitudes across the dataset
shap.summary_plot(
    shap_values, X, plot_type="bar",
    class_names = le.classes_,
    plot_size = (10,6)
)

In the cell below, we show the SHAP values separately by class. We only show the top 10 features.

In [ ]:
avg_shap_class = [
    pd.Series(
        np.abs(shap_values[i]).mean(0),
        index = X.columns.tolist()
    ).sort_values(ascending=True) for i in range(3)
]

fig, axs = plt.subplots(1, 3, figsize=(15,4), dpi=150)
for i in range(3):
    _ = avg_shap_class[i].iloc[-10:].plot(kind='barh', ax=axs[i])
    _ = axs[i].set_title(f'Class: {le.classes_[i]}')
    
fig.tight_layout()

### Dependence plots

In the cell below, we plot the SHAP dependence plots for the top 3 features for each class. 

In [ ]:
n_features = 3
for i in range(3):
    fig, axs = plt.subplots(1, n_features, figsize=(5*n_features,4), dpi=100)
    
    features = avg_shap_class[i].iloc[-n_features:].iloc[::-1].index.tolist()
    
    for j, feature in enumerate(features):
        _ = shap.dependence_plot(
            feature, shap_values[i], X, 
            interaction_index= None, alpha=0.7,
            ax = axs[j], show=False
        )
        if X[feature].dtype == 'category':
            _ = axs[j].tick_params(axis='x', rotation=60)
        
    fig.suptitle(f'SHAP dependence plots for class={le.classes_[i]}')
    fig.tight_layout(rect=[0,0,1,0.99])
    fig.savefig(f'SHAP_dependence_{le.classes_[i]}',bbox_inches='tight')
    fig.show()

## Model with fewer features

We will now consider a model with the reduced number of features. The selected features occur in the set of top 10 features for atleast one of the three classes.

In [ ]:
reduced_features = set()
for i in range(3):
    reduced_features = reduced_features.union(
        set(avg_shap_class[i].iloc[-10:].index.tolist())
    )
    
reduced_features = list(reduced_features)
print(f'Number of feature selected: {len(reduced_features)}')
print('List of selected features:')
print(reduced_features)

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
cv_f1_reduced_scores = [None]*40
for i, (train_index, test_index) in tqdm(enumerate(cv.split(X,y))):
    cv_f1_reduced_scores[i] = fit_and_test_fold(X[reduced_features], y, train_index, test_index)

cv_f1_reduced = np.mean(cv_f1_reduced_scores)
print(f'CV F1 for model with reduced number of features: {cv_f1_reduced:.3f}')

In [ ]:
# CV F1 for each replicate of 10-fold CV
# Clearly, there is some variability across replicates
np.array(cv_f1_reduced_scores).reshape(-1,10).mean(-1)

In [ ]:
model_reduced = fit_model(X[reduced_features], y, n_jobs=4, verbose=50, random_seed=12)
# save models to disk
model_reduced.save_model('reduced_feats.cbm',format='cbm')

## Test predictions

In [ ]:
submission = pd.DataFrame({
    'id':test.index.values,
    'outcome':le.inverse_transform(model.predict(test).ravel())
})
submission.to_csv('submission_orig.csv',index=False)
submission['outcome'].value_counts()/submission.shape[0]

In [ ]:
submission = pd.DataFrame({
    'id':test.index.values,
    'outcome':le.inverse_transform(model_reduced.predict(test[reduced_features]).ravel())
})
submission.to_csv('submission_reduced.csv',index=False)
submission['outcome'].value_counts()/submission.shape[0]